In [ ]:
Hola Chat, vamos a hacer el algoritmo de berkeley
Para esto tenemos un archivo llamado "servidor.py" que contiene el objeto del servidor
otro archivo llamado "cliente.py" que contiene el codigo del objeto cliente
y un archivo llamado "berkeley.py" que contiene la logica para realizar la organizacion del algoritmo:
    En este ultimo archivo se encuentra la logica para realizar el algoritmo de berkeley
    iniciando con la instancia de 3 clientes con sus respectivos relojes que sean con la hora actual +- 0.5 a 0.01 segundos
    y un servidor que se encargara de realizar la sincronizacion de los relojes de los clientes
    Este mismo archivo "berkeley.py" inicia un blucle donde reaalizara la orquestacion de comunicacion entre los clientes y el servidor
    para realizar la sincronizacion de los relojes de los clientes al menos 3 veces
    Se tiene que mostrar en la terminal el proceso paso a paso de la sincronizacion de los relojes de los clientes
    El archibo "berkeley.py" debe encargarse de instaciar cada objeto en el orden correcto y de realizar la comunicacion entre los objetos


Te paso lo que llevo a continuacion para que te sirva de inspiracion y puedas terminar el algoritmo de berkeley
archivo "servidor.py":
import socket
import threading

class Servidor:
    def __init__(self, host, port):
        self.host = host  # Dirección IP del servidor
        self.port = port  # Puerto en el que el servidor escuchará las conexiones
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Crear un socket TCP/IP

    def iniciar(self):
        self.server_socket.bind((self.host, self.port))  # Asociar el socket con la dirección y el puerto
        self.server_socket.listen(5)  # Escuchar conexiones entrantes, permitir hasta 5 conexiones en cola
        print("Esperando conexiones...")
        while True:
            client_socket, client_address = self.server_socket.accept()  # Aceptar la conexión entrante
            print(f"(servidor): Conexión establecida con {client_address}")
            cliente = Cliente(client_socket, self)  # Crear un objeto Cliente para manejar la conexión con el cliente
            recibir_thread = threading.Thread(target=cliente.recibir_mensajes)
            recibir_thread.start()  # Iniciar un hilo para manejar la recepción de mensajes del cliente
            enviar_thread = threading.Thread(target=self.enviar_mensajes, args=(cliente,))
            enviar_thread.start()  # Iniciar un hilo para manejar el envío de mensajes al cliente

    def enviar_mensajes(self, cliente):
        while True:
            mensaje_enviado = input( )  # Leer mensaje desde la consola
            cliente.enviar_mensaje(mensaje_enviado)  # Enviar el mensaje al cliente

class Cliente:
    def __init__(self, client_socket, servidor):
        self.client_socket = client_socket  # Socket del cliente
        self.servidor = servidor  # Instancia del servidor asociado

    def recibir_mensajes(self):
        while True:
            try:
                mensaje_recibido = self.client_socket.recv(1024).decode('utf-8')  # Recibir mensaje del cliente
                print(f"(cliente): {mensaje_recibido}")  # Imprimir el mensaje recibido
            except OSError:
                break

    def enviar_mensaje(self, mensaje):
        self.client_socket.send(bytes(mensaje, 'utf-8'))  # Enviar mensaje al cliente

def main():
    host = '127.0.0.1'  # Dirección IP del servidor
    port = 9999  # Puerto en el que el servidor escuchará las conexiones
    servidor = Servidor(host, port)  # Crear una instancia de Servidor
    servidor.iniciar()  # Iniciar el servidor

if __name__ == "__main__":
    main()



archivo "cliente.py":
import socket
import threading

class Cliente:
    def __init__(self, host, port):
        self.host = host  # Dirección IP del servidor
        self.port = port  # Puerto en el que el servidor está escuchando
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Crear un socket TCP/IP
        self.conectar()  # Conectarse al servidor

    def conectar(self):
        self.client_socket.connect((self.host, self.port))  # Establecer la conexión con el servidor
        recibir_thread = threading.Thread(target=self.recibir_mensajes)
        recibir_thread.start()  # Iniciar un hilo para manejar la recepción de mensajes del servidor
        print("Conectado al servidor. Puedes empezar a enviar mensajes.")

    def recibir_mensajes(self):
        while True:
            try:
                mensaje_recibido = self.client_socket.recv(1024).decode('utf-8')  # Recibir mensaje del servidor
                print(f"(servidor): {mensaje_recibido}")  # Imprimir el mensaje recibido
            except OSError:
                break

    def enviar_mensaje(self, mensaje):
        self.client_socket.send(bytes(mensaje, 'utf-8'))  # Enviar mensaje al servidor

def main():
    host = '127.0.0.1'  # Dirección IP del servidor
    port = 9999  # Puerto en el que el servidor está escuchando
    cliente = Cliente(host, port)  # Crear una instancia de Cliente
    while True:
        mensaje = input( )  # Leer mensaje desde la consola
        cliente.enviar_mensaje(mensaje)  # Enviar el mensaje al servidor

if __name__ == "__main__":
    main()



archivo "berkeley.py":
import time
from cliente import Cliente

class BerkeleyClient(Cliente):
    def __init__(self, host, port):
        super().__init__(host, port)
        self.time_offset = 0

    def adjust_time(self, new_time):
        self.time_offset = new_time - time.time()

    def get_time(self):
        return time.time() + self.time_offset

def main():
    host = '127.0.0.1'
    port = 9999

    # Crear 3 clientes
    clientes = [BerkeleyClient(host, port) for _ in range(3)]

    # Supongamos que el primer cliente es el maestro
    maestro = clientes[0]

    while True:
        # El maestro recoge los tiempos de todos los clientes
        times = [cliente.get_time() for cliente in clientes]

        # El maestro calcula el tiempo medio
        average_time = sum(times) / len(times)

        # El maestro envía el tiempo medio a todos los clientes
        for cliente in clientes:
            cliente.adjust_time(average_time)

        time.sleep(1)  # Esperar un segundo antes de la próxima sincronización

if __name__ == "__main__":
    main()
